<a href="https://colab.research.google.com/github/erindakapllani/question_generator/blob/main/2407_qa_pair_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import files
uploaded = files.upload()


Saving qa_pairs.csv to qa_pairs.csv


In [38]:
import pandas as pd

# Try different encodings
encodings = ['utf-8', 'ISO-8859-1', 'utf-16']

# Attempt to read the file with different encodings
for encoding in encodings:
    try:
        df = pd.read_csv('qa_pairs.csv', encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to read the file with encoding: {encoding}")

# Convert DataFrame to a list of dictionaries
qa_pairs = df.to_dict(orient='records')


Failed to read the file with encoding: utf-8
Successfully read the file with encoding: ISO-8859-1


In [41]:
# Convert DataFrame to a list of dictionaries
qa_pairs = df.to_dict(orient='records')

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example text chunks
text_chunks = [
    "Effects predictions should consider the effects of climate and environmental change on populations of non-human biota that could adversely alter predicted environmental effects due to site activities or introduce new potential environmental effects.",
    "The mitigation plans for prevention or reduction of plant intake fouling should take into account projected effects of climate change, including frazil ice and bio-fouling (mussels, algae, marine plants).",
    "Future meteorological conditions (that is, accounting for climate change) and the extent of thermal plume from modelling should be used as a basis for extrapolating the long-term ice conditions / silt / fish / mussel / algae density observations for source water body and future potential for effects on the project."
]

def check_relevance(chunk, qa_pair):
    """
    Check if the answer in the QA pair is relevant to the question within the context of the chunk
    using semantic similarity.
    """
    question = qa_pair['Questions']
    answer = qa_pair['Answers']

    # Encode texts
    chunk_embedding = model.encode(chunk, convert_to_tensor=True)
    question_embedding = model.encode(question, convert_to_tensor=True)
    answer_embedding = model.encode(answer, convert_to_tensor=True)

    # Calculate cosine similarities
    question_similarity = util.pytorch_cos_sim(question_embedding, chunk_embedding).item()
    answer_similarity = util.pytorch_cos_sim(answer_embedding, chunk_embedding).item()

    # Determine relevance (this threshold can be adjusted)
    threshold = 0.5
    if question_similarity > threshold and answer_similarity > threshold:
        return True
    else:
        return False

# Randomly select a chunk and cross-check the QA pairs
random_chunk = random.choice(text_chunks)

print(f"Selected Chunk:\n{random_chunk}\n")

for qa in qa_pairs:
    is_relevant = check_relevance(random_chunk, qa)
    print(f"Question: {qa['Questions']}")
    print(f"Answer: {qa['Answers']}")
    print(f"Relevant: {is_relevant}\n")

Streaming output truncated to the last 5000 lines.
Question: <question>Are perimeter fences important when establishing a construction site for a nuclear reactor?</question>
Answer: <answer>Yes, perimeter fences are important when establishing a construction site for a nuclear reactor as they help in securing the site and controlling access to authorized personnel only.</answer>
Relevant: False

Question: <question>What physical protection considerations may be different for reactor facilities located in a remote area compared to those in a large urban area?</question>
Answer: <answer>Reactor facilities located in a remote area bordered by a small population density may require different physical protection considerations than those that apply to reactor facilities located in a large urban area.</answer>
Relevant: False

Question: <question>What should the applicant evaluate when considering remote sites for reactor facilities?</question>
Answer: <answer>The applicant should evaluate r